In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data

import torch.optim as optim

import torchvision
from torchvision import datasets as dsets
from torchvision import transforms

import torchvision.models.resnet as resnet

from matplotlib.pyplot import imshow
%matplotlib inline

import visdom
import math
import vgg
import numpy as np

In [ ]:
vis = visdom.Visdom()
vis.close(env="main")

In [ ]:
def plt_tracker(loss_plot, loss_value, num):
    vis.line(X=np.stack(np.arange(num, num+1)),
             Y=np.stack([loss_value]),
             win=loss_plot,
             update='append'
             )

In [ ]:
if (torch.cuda.is_available() == 1):
    print("cuda is available")
    device = 'cuda'
else:
    device = 'cpu'

In [ ]:
transform = transforms.Compose([
    #transforms.Resize(256),
    transforms.RandomCrop(56),
    transforms.ToTensor(),]
    #transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))]
)

In [ ]:
try:
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    print("Dataset is alreay downlaoded")
except:
    !./download.sh
    train = dsets.ImageFolder(root='./tiny-imagenet-200/train',transform=transform)
    
        
validation = dsets.ImageFolder(root='./tiny-imagenet-200/val',transform=transform)
test = dsets.ImageFolder(root='./tiny-imagenet-200/test',transform=transform)

print(train)
print(validation)
print(test)


check_img=train.__getitem__(0)
ToPIL=transforms.ToPILImage()
check_img = ToPIL(check_img[0])
print(check_img)

imshow(check_img)

In [ ]:
train_loader=data.DataLoader(train, batch_size=256, shuffle=True, num_workers=2)
var_loader=data.DataLoader(validation, batch_size=64, shuffle=False, num_workers=2)
test_loader=data.DataLoader(test, batch_size=64, shuffle=False, num_workers=2)

In [ ]:
BasicBlock = resnet.BasicBlock
Bottleneck = resnet.Bottleneck

class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        #x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
CNN = ResNet(BasicBlock, [2,2,2,2])
CNN=CNN.to(device)
#print(CNN)

In [ ]:
test_input = torch.Tensor(3,3,56,56).to(device)

out = CNN(test_input)
print(out.shape)

In [ ]:
optimizer = optim.SGD(CNN.parameters(),lr=0.01,momentum=0.9)

loss_function = nn.CrossEntropyLoss()

lr_sche = optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma=0.8)

In [ ]:
loss_plot = vis.line(Y=torch.Tensor(1).zero_())

In [ ]:
epochs = 30
print("length of train_loader : {}".format(len(train_loader)))
for epoch in range(epochs):
    running_loss = 0.0
    check_loss = 0.0
    lr_sche.step()
    for num, data in enumerate(train_loader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = CNN(inputs)
        loss = loss_function(out, labels)
        loss.backward()
        
        optimizer.step()
        running_loss += loss.item()
        check_loss += loss.item()
        
        if (num % 30 == 1) and (num != 1):
            check_loss = check_loss/30
            print("loss : {}".format(check_loss))
            plt_tracker(loss_plot,(check_loss), epoch *len(train_loader) + num)
            check_loss = 0  
        
        if num % len(train_loader) == (len(train_loader) - 1):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, num + 1, running_loss / len(train_loader)))
            running_loss = 0.0
            check_loss = 0.0

In [ ]:
torch.save(CNN.state_dict(),"./model/tiny_imagenet_resnet.pth")

In [ ]:
load_CNN = ResNet(BasicBlock, [2,2,2,2])
load_CNN=CNN.to(device)

In [ ]:
load_CNN.load_state_dict(torch.load('./model/tiny_imagenet_resnet.pth',map_location = device))
load_CNN = load_CNN.to(device)

In [ ]:
print(load_CNN.conv1.weight[0][0])


In [ ]:
print(CNN.conv1.weight.data[0][0])


In [ ]:
load_CNN == CNN